# QC assessment of NGS data
As mentioned previously, QC is an important part of any analysis. In this section we are going to look at some of the metrics and graphs that can be used to assess the QC of NGS data. 

Before we start, change into the `QC/qc-training-notebooks/` directory:

In [ ]:
cd ~/QC/qc-training-notebooks

## Base quality
[Illumina sequencing](https://en.wikipedia.org/wiki/Illumina_dye_sequencing) technology relies on sequencing by synthesis. One of the most common problems with this is __dephasing__. For each sequencing cycle, there is a possibility that the replication machinery slips and either incorporates more than one nucleotide or perhaps misses to incorporate one at all. The more cycles that are run (i.e. the longer the read length gets), the greater the accumulation of these types of errors gets. This leads to a heterogeneous population in the cluster, and a decreased signal purity, which in turn reduces the precision of the base calling. The figure below shows an example of this.

![Base Quality](img/base_qual.png)

Because of dephasing, it is possible to have high-quality data at the beginning of the read but really low-quality data towards the end of the read. In those cases you can decide to trim off the low-quality reads, for example using a tool called [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic).

The figures below shows an example of a high-quality read data (top) and a poor quality read data (bottom).

![High-quality read data](img/base_qual_pass.png)

![Poor quality read data](img/base_qual_fail.png)

## Other base calling errors
There are several different reasons for a base to be called incorrectly, as shown in the figure below. __Phasing noise__ and __signal decay__ is a result of the dephasing issue described above. 
During library preparation, __mixed clusters__ can occur if multiple templates get co-located. These clusters should be removed from the downstream analysis. 
__Boundary effects__ occur due to optical effects when the intensity is uneven across each tile, resulting in higher intensity found toward the center. 
__Cross-talk__ occurs because the emission frequency spectra for each of the four base dyes partly overlap, creating uncertainty. 
Finally, for previous sequencing cycle methods __T fluorophore accumulation__ was an issue, where incomplete removal of the dye coupled to thymine lead to an ambient accumulation the nucleotides, causing a false high Thymine trend.

![Base Calling Errors](img/base_calling_errors.jpg)

_Base-calling for next-generation sequencing platforms_, doi: [10.1093/bib/bbq077](https://academic.oup.com/bib/article/12/5/489/268399)

## Mismatches per cycle
Aligning reads to a high-quality reference genome can provide insight to the quality of a sequencing run by showing you the mismatches to the reference sequence. This can help you detect cycle-specific errors. Mismatches can occur due to two main causes, sequencing errors and differences between your sample and the reference genome, which is important to bear in mind when interpreting mismatch graphs. The figures below show an example of a good run (top) and a bad one (bottom). In the first figure, the distribution of the number of mismatches is even between the cycles, which is what we would expect from a good run. However, in the second figure, two cycles stand out with a lot of mismatches compared to the other cycles.

![Good run](img/mismatch_per_cycle_pass.png)

![Poor run](img/mismatch_per_cycle_fail.png)

## GC bias
It is a good idea to compare the GC content of the reads against the expected distribution in a reference sequence. The GC content varies between species, so a shift in GC content like the one seen below could be an indication of sample contamination. In the left image below, we can see that the GC content of the sample is about the same as for the reference, at ~38%. However, in the right figure, the GC content of the sample is closer to 55%, indicating that there is an issue with this sample. 

![GC Bias](img/gc_bias.png)

## GC content by cycle
Looking at the GC content per cycle can help detect if the adapter sequence was trimmed. For a random library, it is expected to be little to no difference between the different bases of a sequence run, so the lines in this plot should be parallel with each other like in the first of the two figures below. In the second of the figures, the initial spikes are likely due to adapter sequences that have not been removed. 

![Good run](img/acgt_per_cycle_pass.png)

![Poor run](img/acgt_per_cycle_fail.png)

## Insert size
For paired-end sequencing the size of DNA fragments also matters. In the first of the examples below, the insert size peaks around 440 bp. In the second however, there is also a peak at around 200 bp. This indicates that there was an issue with the fragment size selection during library prep.

![Good run](img/insert_size_pass.png)

![Poor run](img/insert_size_fail.png)

### Exercises
__Q1: The figure below is from a 100bp paired-end sequencing. Can you spot any problems?__

![Q1](img/insert_size_quiz.png)

## Insertions/Deletions per cycle
Sometimes, air bubbles occur in the flow cell, which can manifest as false indels. The spike in the second image provides an example of how this can look.

![Good run](img/indels-per-cycle.pass.png)

![Poor run](img/indels-per-cycle.fail.png)

## Generating QC stats
Now let's try this out! We will generate QC stats for two lanes of Illumina paired-end sequencing data from yeast. We will use the bwa mapper to align the data to the [Saccromyces cerevisiae genome](ftp://ftp.ensembl.org/pub/current_fasta/saccharomyces_cerevisiae/dna), followed by samtools stats to generate the stats.

Read pairs are usually stored in two separate FASTQ files so that n-th read in the first file and the n-th read in the second file constitute a read pair. Can you devise a quick sanity check that reads in these two files indeed form pairs? The files must have the same number of lines and the naming of the reads usually suggests if they form a pair. The location of the files is:  
```  
data/lane1/s_7_1.fastq   
data/lane1/s_7_2.fastq  
```

Let's have a look at the script we are going to run to create the mappings:

In [ ]:
cat create_mapping.sh

The script contains several commands, some are combined together using pipes. (UNIX pipes is a very powerful and elegant concept which allows us to feed the output of one command into the next command and avoid writing intermediate files. If you are not comfortable with UNIX, consider having a go at the UNIX tutorial).

Now run the script to create the mappings and stats (this may take a while; the message `Processed 27130 reads...` will appear when it is done):

In [ ]:
./create_mapping.sh

The script will produce the BAM file lane1.sorted.bam and a matching index file: 

In [ ]:
ls -alrt data

Now we will use __`samtools stats`__ to generate the stats for the primary alignments. The option __`-f`__ can be used to filter reads with specific tags, while __`-F`__ can be used to _filter out_ reads with specific tags. The following command will include only primary alignments:

In [ ]:
samtools stats -F SECONDARY data/lane1.sorted.bam \
    > data/lane1.sorted.bam.bchk

Have a look at the first 47 lines of the statistics file that was generated:

In [ ]:
head -n 47 data/lane1.sorted.bam.bchk

This file contains a number of useful stats that we can use to get a better picture of our data, and it can even be plotted with __`plot-bamstats`__, as you will see soon. First let's have a closer look at some of the different stats. Each part of the file starts with a `#` followed by a description of the section and how to extract it from the file. Let's have a look at all the sections in the file:

In [ ]:
grep ^'#' data/lane1.sorted.bam.bchk | grep 'Use'

### Summary Numbers (SN)
This initial section contains a summary of the alignment and includes some general statistics. In particular, you can see how many bases mapped, and how much of the genome that was covered.

Now look at the output and try to answer the questions below.

__Q2: What is the total number of reads?__

__Q3: What proportion of the reads were mapped?__

__Q4: How many pairs were mapped to a different chromosome?__

__Q5: What is the insert size mean and standard deviation?__

__Q6: How many reads were paired properly?__

Finally, we will create some QC plots from the output of the stats command using the command __plot-bamstats__ which is included in the samtools package: 

In [ ]:
plot-bamstats -p data/lane1-plots/ data/lane1.sorted.bam.bchk

plot-bamstats creates a file index.html in the output doirectory (data/lane1-plots/) which you can open in a web browser to view the QC information.
Unfortunately this does not display correctly if you open it in JupterHub, but this is what you would see:

<table style="vertical-align:middle; padding: 0.5em; border: 1px solid black; border-collapse:collapse; margin-left:20px;">
   <tr style="background-color:#ffffff;">
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/insert-size.png"><span>Insert size<br><img src="data/lane1-plots/insert-size.png" width="200px"></img></span></a>
      </td>
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/gc-content.png"><span>GC content<br><img src="data/lane1-plots/gc-content.png" width="200px"></img></span></a>
      </td>
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/acgt-cycles.png"><span>Per-base sequence content<br><img src="data/lane1-plots/acgt-cycles.png" width="200px"></img></span></a>
      </td>
   </tr>
   <tr style="background-color:#ffffff;">
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         &nbsp;
      </td>
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/quals.png"><span>Quality per cycle<br><img src="data/lane1-plots/quals.png" width="200px"></img></span></a>
      </td>
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/quals2.png"><span>Quality per cycle<br><img src="data/lane1-plots/quals2.png" width="200px"></img></span></a>
      </td>
   </tr>
   <tr style="background-color:#ffffff;">
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/quals3.png"><span>Quality per cycle<br><img src="data/lane1-plots/quals3.png" width="200px"></img></span></a>
      </td>
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/quals-hm.png"><span>Quality per cycle<br><img src="data/lane1-plots/quals-hm.png" width="200px"></img></span></a>
      </td>
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/indel-cycles.png"><span>Indels per cycle<br><img src="data/lane1-plots/indel-cycles.png" width="200px"></img></span></a>
      </td>
   </tr>
   <tr style="background-color:#ffffff;">
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/indel-dist.png"><span>Indel lengths<br><img src="data/lane1-plots/indel-dist.png" width="200px"></img></span></a>
      </td>
      <td style="vertical-align:middle; padding: 0.5em; border: 1px solid black;">
         <a style="text-decoration:none; color:black; font-weight:bold;" href="data/lane1-plots/gc-depth.png"><span>Mapped depth vs GC<br><img src="data/lane1-plots/gc-depth.png" width="200px"></img></span></a>
      </td>
   </tr>
</table>

<table style="text-align: left; margin-top: 1em; margin-left:20px; border: 1px dotted #83A4C3; background-color: #F5F5F5; padding: 0.5em;">
   <tr>
      <th style="text-align: left; background-color: #F5F5F5;">
         Reads
      </th>
   </tr>
   <tr>
      <td style="padding-left:1em; vertical-align:top;">
         <table style="text-align: left; background-color: #F5F5F5; padding: 0.5em;">
            <tr style="background-color: #F5F5F5;"><td>total:       <td style="text-align:right; padding-left:1em; "> 400,252    </td><td style="text-align:right; padding-left:1em; "></td></tr>
            <tr style="background-color: #F5F5F5;"><td>filtered:    <td style="text-align:right; padding-left:1em; "> 2,746 </td><td style="text-align:right; padding-left:1em; "> (0.7%)</td></tr>
            <tr style="background-color: #F5F5F5;"><td>non-primary: <td style="text-align:right; padding-left:1em; "> 0  </td><td style="text-align:right; padding-left:1em; "> </td></tr>
            <tr style="background-color: #F5F5F5;"><td>duplicated:  <td style="text-align:right; padding-left:1em; "> 0      </td><td style="text-align:right; padding-left:1em; "> (0.0%)</td></tr>
            <tr style="background-color: #F5F5F5;"><td>mapped:      <td style="text-align:right; padding-left:1em; "> 303,036   </td><td style="text-align:right; padding-left:1em; "> (76.2%)</td></tr>
            <tr style="background-color: #F5F5F5;"><td>zero MQ:     <td style="text-align:right; padding-left:1em; "> 23,803      </td><td style="text-align:right; padding-left:1em; "> (7.9%)</td></tr>
            <tr style="background-color: #F5F5F5;"><td>avg read length: <td style="text-align:right; padding-left:1em; "> 108    </td><td style="text-align:right; padding-left:1em; "></td></tr>
         </table>
      </td>
   </tr>
   <tr>
      <th style="text-align: left; background-color: #F5F5F5;">
         Bases
      </th>
   </tr>
   <tr>
      <td style="padding-left:1em; vertical-align:top;">
         <table style="text-align: left; background-color: #F5F5F5; padding: 0.5em;">
            <tr style="background-color: #F5F5F5;"><td>total:       <td style="text-align:right; padding-left:1em; "> 42,930,648    </td><td style="text-align:right; padding-left:1em; "></td></tr>
            <tr style="background-color: #F5F5F5;"><td>mapped:      <td style="text-align:right; padding-left:1em; "> 31,886,194   </td><td style="text-align:right; padding-left:1em; "> (74.3%)</td></tr>
            <tr style="background-color: #F5F5F5;"><td>error rate:      <td style="text-align:right; padding-left:1em; "> 1.16%  </td><td style="text-align:right; padding-left:1em; "></td></tr>
         </table>
      </td>
   </tr>
</table>


__Q7: How many reads have zero mapping quality?__

__Q8: Which read (forward/reverse) of the first fragments and second fragments are higher base quality on average?__  

The answers to the questions on this page can be found [here](assessment-answers.ipynb).   

Now continue to the next section of the tutorial: [Identifying contamination](contamination.ipynb)  
Alternatively, you can return to the [previous section](conversion.ipynb) or the [index page](index.ipynb).